# Lab 06: CSP Solutions

# Task 1


In [ ]:
#Import the google ortools library to solve the csp problem 
from ortools.sat.python import cp_model

N = 5
start = (1, 1)
end = (4, 4)

obstacles = {(2, 2)}

diagonal_moves = [(-1, -1), (-1, 1), (1, -1), (1, 1)]

max_steps = 10

model = cp_model.CpModel()

x = [model.NewIntVar(0, N - 1, f'x_{i}') for i in range(max_steps)]
y = [model.NewIntVar(0, N - 1, f'y_{i}') for i in range(max_steps)]

model.Add(x[0] == start[0] - 1)
model.Add(y[0] == start[1] - 1)
model.Add(x[-1] == end[0] - 1)
model.Add(y[-1] == end[1] - 1)

for i in range(max_steps):
    for (ox, oy) in obstacles:
        model.AddForbiddenAssignments([x[i], y[i]], [(ox - 1, oy - 1)])

for i in range(max_steps - 1):
    dx = model.NewIntVar(-1, 1, f'dx_{i}')
    dy = model.NewIntVar(-1, 1, f'dy_{i}')
    model.Add(dx == x[i + 1] - x[i])
    model.Add(dy == y[i + 1] - y[i])
    model.AddAbsEquality(model.NewIntVar(1, 1, ''), dx)
    model.AddAbsEquality(model.NewIntVar(1, 1, ''), dy)

for i in range(max_steps):
    for j in range(i + 1, max_steps):
        model.Add(x[i] != x[j]).OnlyEnforceIf(y[i] == y[j])
        model.Add(y[i] != y[j]).OnlyEnforceIf(x[i] == x[j])

active = [model.NewBoolVar(f'active_{i}') for i in range(max_steps)]
for i in range(max_steps):
    is_target = model.NewBoolVar(f'is_target_{i}')
    model.Add(x[i] == end[0] - 1).OnlyEnforceIf(is_target)
    model.Add(y[i] == end[1] - 1).OnlyEnforceIf(is_target)
    model.Add(active[i] == is_target)

model.Maximize(sum(active))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Path:")
    for i in range(max_steps):
        px = solver.Value(x[i])
        py = solver.Value(y[i])
        print(f"Step {i + 1}: ({px + 1}, {py + 1})")
        if (px + 1, py + 1) == end:
            break
else:
    print("No path found.")


# Task 2


In [ ]:
from ortools.sat.python import cp_model
from collections import deque

grid = [
    [0, 1, 1, 0, 0],
    [1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 1, 1, 0, 0]
]

rows = len(grid)
cols = len(grid[0])

dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def largest_landmass(grid):
    visited = [[False]*cols for _ in range(rows)]
    max_region = set()

    def bfs(r, c):
        q = deque()
        region = set()
        q.append((r, c))
        visited[r][c] = True
        while q:
            x, y = q.popleft()
            region.add((x, y))
            for dx, dy in dirs:
                nx, ny = x + dx, y + dy
                if (0 <= nx < rows and 0 <= ny < cols and 
                    grid[nx][ny] == 1 and not visited[nx][ny]):
                    visited[nx][ny] = True
                    q.append((nx, ny))
        return region

    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 1 and not visited[i][j]:
                region = bfs(i, j)
                if len(region) > len(max_region):
                    max_region = region
    return max_region

land_cells = largest_landmass(grid)

model = cp_model.CpModel()
perimeter = model.NewIntVar(0, 4 * len(land_cells), 'perimeter')

perimeter_terms = []

for x, y in land_cells:
    for dx, dy in dirs:
        nx, ny = x + dx, y + dy
        if (nx < 0 or ny < 0 or nx >= rows or ny >= cols or (nx, ny) not in land_cells):
            perimeter_terms.append(1)

model.Add(perimeter == sum(perimeter_terms))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("Perimeter of largest landmass:", solver.Value(perimeter))
else:
    print("No solution found.")


# Task 3

In [ ]:
import itertools

distance = [
    [0, 29, 20, 21, 17, 18, 23, 22, 19, 28],
    [29, 0, 15, 29, 28, 40, 72, 21, 29, 25],
    [20, 15, 0, 15, 14, 25, 81, 10, 12, 23],
    [21, 29, 15, 0, 4, 12, 92, 12, 25, 13],
    [17, 28, 14, 4, 0, 16, 94, 9, 20, 16],
    [18, 40, 25, 12, 16, 0, 95, 24, 36, 3],
    [23, 72, 81, 92, 94, 95, 0, 90, 101, 99],
    [22, 21, 10, 12, 9, 24, 90, 0, 15, 25],
    [19, 29, 12, 25, 20, 36, 101, 15, 0, 35],
    [28, 25, 23, 13, 16, 3, 99, 25, 35, 0]
]

cities = list(range(10))
min_cost = float('inf')
best_path = []

for perm in itertools.permutations(cities):
    cost = 0
    for i in range(len(perm) - 1):
        cost += distance[perm[i]][perm[i+1]]
    cost += distance[perm[-1]][perm[0]]  # Return to start
    if cost < min_cost:
        min_cost = cost
        best_path = perm

print("Best path:", best_path)
print("Minimum cost:", min_cost)


# Task 4


In [ ]:
#Import the google ortools library to solve the csp problem 
from ortools.sat.python import cp_model

def warehouse_csp():
    grid_size = 6
    num_robots = 5
    num_packages = 10
    max_time = 20
    max_capacity = 10
    package_weights = [1, 3, 2, 2, 1, 4, 3, 1, 2, 2]
    delivery_points = [(5, 5), (1, 1), (2, 2), (3, 3), (0, 5),
                       (4, 2), (2, 4), (5, 0), (0, 0), (3, 5)]
    package_locations = [(0, 1), (1, 3), (2, 5), (3, 1), (4, 0),
                         (5, 3), (1, 2), (3, 2), (2, 0), (4, 4)]
    charging_stations = [(0, 0), (5, 5)]

    model = cp_model.CpModel()

    robot_pos = {}
    for r in range(num_robots):
        for t in range(max_time):
            robot_pos[r, t] = model.NewIntVar(0, grid_size * grid_size - 1, f'pos_r{r}_t{t}')

    package_status = {}
    for p in range(num_packages):
        for t in range(max_time):
            package_status[p, t] = model.NewIntVar(0, 2, f'status_p{p}_t{t}')

    for t in range(max_time):
        model.AddAllDifferent([robot_pos[r, t] for r in range(num_robots)])

    for p in range(num_packages):
        model.Add(package_status[p, max_time - 1] == 2)

    model.Minimize(sum(package_status[p, t] != 2 for p in range(num_packages) for t in range(max_time)))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution found!")
        for r in range(num_robots):
            print(f"\nRobot {r} path:")
            for t in range(max_time):
                pos = solver.Value(robot_pos[r, t])
                x, y = divmod(pos, grid_size)
                print(f"  t={t}: ({x}, {y})")
    else:
        print("No solution found.")

warehouse_csp()


# Task 5


In [ ]:
#Import the google ortools library to solve the csp problem 
from ortools.sat.python import cp_model

def is_prime(n):
    return n in [2, 3, 5, 7]

def sudoku_solver_with_constraints(grid):
    model = cp_model.CpModel()
    cell = {}

    for i in range(9):
        for j in range(9):
            cell[i, j] = model.NewIntVar(1, 9, f'cell_{i}_{j}')

    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                model.Add(cell[i, j] == grid[i][j])

    for i in range(9):
        model.AddAllDifferent([cell[i, j] for j in range(9)])
        model.AddAllDifferent([cell[j, i] for j in range(9)])

    for row in range(0, 9, 3):
        for col in range(0, 9, 3):
            model.AddAllDifferent([cell[row + i, col + j] for i in range(3) for j in range(3)])

    main_diag = [cell[i, i] for i in range(9)]
    anti_diag = [cell[i, 8 - i] for i in range(9)]
    model.Add(sum(main_diag) % 3 == 0)
    model.Add(sum(anti_diag) % 3 == 0)

    primes = [2, 3, 5, 7]
    for i in range(9):
        for j in range(9):
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i + dx, j + dy
                if 0 <= ni < 9 and 0 <= nj < 9:
                    for p1 in primes:
                        for p2 in primes:
                            b1 = model.NewBoolVar('')
                            b2 = model.NewBoolVar('')
                            model.Add(cell[i, j] == p1).OnlyEnforceIf(b1)
                            model.Add(cell[ni, nj] == p2).OnlyEnforceIf(b2)
                            model.AddBoolAnd([b1, b2]).OnlyEnforceIf(model.NewBoolVar('false')).Not()

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        for i in range(9):
            print([solver.Value(cell[i, j]) for j in range(9)])
    else:
        print("No solution found.")

sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

sudoku_solver_with_constraints(sudoku_grid)
